# MAG analysis notebook

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from MAG import MicrosoftAcademicGraph
import os
from sparkhpc import sparkjob
import findspark

# set environment variables
os.environ["SPARK_LOCAL_DIRS"] = "/home/laal/MAG/TMP"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.8.0-openjdk-1.8.0.242.b08-0.el7_7.x86_64"
os.environ['SPARK_HOME'] = "/home/laal/MAG/spark-3.0.2-bin-hadoop2.7"

In [2]:
sj = sparkjob.sparkjob(jobid=36492, memory_per_executor=15500)

In [3]:
job = sj.start_spark()

['NAME STATE JOBID', 'hebbian_weights_submit PENDING 36481', 'roberta_base RUNNING 36230', 'simple RUNNING 35764', 'sparkcluster RUNNING 36492', 'hebbian_weights_submit RUNNING 36236', 'hebbian_weights_submit RUNNING 36235', 'jupyter RUNNING 36477', 'bikenwgrowth RUNNING 36387_46', 'bikenwgrowth RUNNING 36387_47', 'bikenwgrowth RUNNING 36387_48', 'bikenwgrowth RUNNING 36387_49', 'bikenwgrowth RUNNING 36387_50', 'bikenwgrowth RUNNING 36387_51', 'bikenwgrowth RUNNING 36387_52', 'bikenwgrowth RUNNING 36387_53', 'bikenwgrowth RUNNING 36387_54', 'bikenwgrowth RUNNING 36387_55', 'bikenwgrowth RUNNING 36387_57', 'bikenwgrowth RUNNING 36387_58', 'bikenwgrowth RUNNING 36387_23', 'bikenwgrowth RUNNING 36387_24', 'bikenwgrowth RUNNING 36387_25', 'bikenwgrowth RUNNING 36387_26', 'bikenwgrowth RUNNING 36387_28', 'bikenwgrowth RUNNING 36387_29', 'bikenwgrowth RUNNING 36387_30', 'bikenwgrowth RUNNING 36387_31', 'bikenwgrowth RUNNING 36387_32', 'bikenwgrowth RUNNING 36387_33', 'bikenwgrowth RUNNING 36

In [4]:
spark = SparkSession.builder.config(conf=job.getConf()).getOrCreate()

In [5]:
mag = MicrosoftAcademicGraph(spark=spark, data_folderpath="/home/laal/MAG/DATA/")

In [6]:
def author_to_field_of_study(mag):
    author_affiliations = mag.getDataframe('PaperAuthorAffiliations')
    authors = mag.getDataframe('WosToMag')
    paper_root_field = mag.getDataframe('PaperRootField')

    query = """
    SELECT paa.AuthorId, 
           prf.AncestorId,
           COUNT(*) AS NumPapersInField
    FROM PaperAuthorAffiliations AS paa
    INNER JOIN WosToMag AS wtm ON paa.AuthorId = wtm.MAG 
    INNER JOIN PaperRootField AS prf ON paa.PaperId = prf.PaperId
    WHERE prf.fieldRank = 1
    GROUP BY paa.AuthorId, 
             prf.AncestorId
    ORDER BY COUNT(*) DESC
    LIMIT 1000
    """
    author_to_field = mag.query_sql(query)
    return author_to_field

In [7]:
author_to_field = author_to_field_of_study(mag)

In [8]:
sj

['NAME STATE JOBID', 'hebbian_weights_submit PENDING 36481', 'roberta_base RUNNING 36230', 'simple RUNNING 35764', 'sparkcluster RUNNING 36492', 'hebbian_weights_submit RUNNING 36236', 'hebbian_weights_submit RUNNING 36235', 'jupyter RUNNING 36477', 'bikenwgrowth RUNNING 36387_46', 'bikenwgrowth RUNNING 36387_47', 'bikenwgrowth RUNNING 36387_48', 'bikenwgrowth RUNNING 36387_49', 'bikenwgrowth RUNNING 36387_50', 'bikenwgrowth RUNNING 36387_51', 'bikenwgrowth RUNNING 36387_52', 'bikenwgrowth RUNNING 36387_53', 'bikenwgrowth RUNNING 36387_54', 'bikenwgrowth RUNNING 36387_55', 'bikenwgrowth RUNNING 36387_57', 'bikenwgrowth RUNNING 36387_58', 'bikenwgrowth RUNNING 36387_23', 'bikenwgrowth RUNNING 36387_24', 'bikenwgrowth RUNNING 36387_25', 'bikenwgrowth RUNNING 36387_26', 'bikenwgrowth RUNNING 36387_28', 'bikenwgrowth RUNNING 36387_29', 'bikenwgrowth RUNNING 36387_30', 'bikenwgrowth RUNNING 36387_31', 'bikenwgrowth RUNNING 36387_32', 'bikenwgrowth RUNNING 36387_33', 'bikenwgrowth RUNNING 36

Job ID,Number of cores,Status,Spark UI,Spark URL,36492,4,None,http://desktop1.hpc.itu.dk:8080,spark://172.16.16.101:7077


In [9]:
job

<SparkContext master=spark://172.16.16.101:7077 appName=pyspark-shell>

In [10]:
spark

In [11]:
author_to_field.show(20)

+----------+----------+----------------+
|  AuthorId|AncestorId|NumPapersInField|
+----------+----------+----------------+
|2310082373| 121332964|            4585|
|2205515583| 121332964|            4448|
|2061787601| 121332964|            4369|
|2759530663| 121332964|            4312|
| 426350050| 121332964|            4144|
|2058546385| 121332964|            4084|
|1796911943| 121332964|            4034|
|2107027955| 121332964|            4029|
|1997479564| 121332964|            4010|
|2145238930| 121332964|            3969|
|2567620095| 121332964|            3945|
|2063723168| 121332964|            3940|
|2077334193| 121332964|            3878|
|2119358905| 121332964|            3845|
|2626143265| 121332964|            3822|
|2055011421| 121332964|            3822|
|2315388596| 121332964|            3816|
|2098153449| 121332964|            3808|
|2094520724| 121332964|            3801|
|2153029437| 121332964|            3790|
+----------+----------+----------------+
only showing top